<a href="https://colab.research.google.com/github/dtabuena/Ephys_DataSets/blob/main/Wild_Type_Excitability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
dataset = {'data_name': 'Wild_Type_Hipp',
           'data_source': "https://www.dropbox.com/scl/fo/xpsvbsuu6nzsl4rtm7fvv/h?rlkey=wj4gdwxalt4wse40um2z2fvjx&dl=1",
           'file_naming_scheme': ['Rec_date','GenoType','Sex','Age','SliceDir','Slice_Num','Cell_num','Cell_Type'],
           }


In [20]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
!pip install openpyxl --quiet
!pip install XlsxWriter --quiet


'''Get Repositories'''
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib
to_import = [
            'ABF_Quality_Control.ipynb',
            'Basic_Ephys.ipynb',
            'Simple_ABF_tools.ipynb',
            'fun_math.ipynb',
            'importing_abfs_from_dropbox.ipynb',
            'QC_recoding_dataframe.ipynb',
            'Analyzers/input_resistance_analyzer.ipynb',
            'Analyzers/gain_analyzer.ipynb',
            'Analyzers/latencey_analyzer.ipynb',
            'Analyzers/IV_analyzer.ipynb',
            'Analyzers/Vm_analyzer.ipynb',
            'Analyzers/membrane_analyzer.ipynb',
            'Analyzers/rheobase_analyzer.ipynb',

            'Ephys_wrapper.ipynb',
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f --quiet


##### Setup Protocol List
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',]
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s'
           'VC - 3min GapFree',
           'I0 - 3min GapFree']

Cloning into 'EphysLib'...
remote: Enumerating objects: 1098, done.
remote: Counting objects: 100% (475/475), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 1098 (delta 349), reused 359 (delta 271), pack-reused 623
Receiving objects: 100% (1098/1098), 17.40 MiB | 15.75 MiB/s, done.
Resolving deltas: 100% (726/726), done.


In [ ]:
data_name = dataset['data_name']
data_source = dataset['data_source']
file_naming_scheme = dataset['file_naming_scheme']

''' Gather and Catalog Source Data'''
file_loc = get_drobox_folder_url(data_source, '' + data_name)
# clear_output(wait=False)
abf_recordings_df, protocol_set = catalogue_recs(file_loc,file_naming_scheme)
print(protocol_set)

In [ ]:
#
Fast_results = ephys_wrapper(dataset,VC_prot,IC_prot,strat_cols=[ 'Cell_Type'])

In [23]:
strat_abfs_by_prot( 'reorg','/content/my_ephys_data_'+dataset['data_name'], f"/content/{dataset['data_name']}_Recording_LookUp.csv" )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
single_val_strat_groups = ['ap_amplitutude',
                            'Rmp_mV',
                            'Ra_160.0',
                            'Rm_160.0',
                            'Cm_pc_10.0',
                            'Cmq_160.0',
                            'Ra_160.0',
                            'Rm_160.0',
                            'Gain_(HzpA)',
                            'max_adapt%',
                            'adapt_thresh_90',
                            'Rheobase',
                            'AP_thresh_US',
                            'fast_after_hyperpol',
                            'Spike_latency_(ms)',
                            'Input_Resistance_MO',
                            ]


manual_exclusions = [''] # Dramatic shift in seal at unknown time during recording.
exclusion_overide = [''] # Vm dramatically improved prior to gain recording


strat_df_dict = Fast_results['strat_df_dict'].copy()
file_naming_scheme = dataset['file_naming_scheme']
filtered_dict, fail_dict = final_qc(strat_df_dict,file_naming_scheme,qc_Rmp=-45,qc_AP_amp=40,qc_RR=.35,qc_Ra=65,manual_exclusions=manual_exclusions,exclusion_overide=exclusion_overide)
alt_strat_dict = restratify_results(filtered_dict,file_naming_scheme,single_val_strat_groups)
resp_curve_list = ['IV_Early_(V_stim)','IV_Early_(I_peak)','IV_Steady_State_(I_mean)','Stim_Levels_(pA)','Spike_Counts']
response_curve_data = stratify_response_curve(filtered_dict,resp_curve_list,strat_list=[''])
alt_strat_dict.update(response_curve_data)

write_strat_dfs(alt_strat_dict, dataset['data_name']+'_results_stratified_alternate')

2023x10x26_WT_F_P586_Sag_s002_c003_CA3 Fail - Rmp = -40.641685485839844 mV > -45 mV
2023x10x30_WT_F_P590_Sag_s001_c001_CA3 Fail - Rmp = -34.172943115234375 mV > -45 mV
2023x10x30_WT_F_P590_Sag_s002_c004_CA3 Fail - Rmp = -41.546669006347656 mV > -45 mV
2023x10x30_WT_F_P590_Sag_s002_c005_CA3 Fail - RR = 5.579830936689313 > 0.35
ap_amplitutude
Rmp_mV
Ra_160.0
Rm_160.0
Cm_pc_10.0
Cmq_160.0
Gain_(HzpA)
max_adapt%
adapt_thresh_90
Rheobase
AP_thresh_US
fast_after_hyperpol
Spike_latency_(ms)
Input_Resistance_MO
_IV_Early_(V_stim)
_IV_Early_(I_peak)
_IV_Steady_State_(I_mean)
_Stim_Levels_(pA)
_Spike_Counts


<ipython-input-20-9455134f582a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(blank_df,ignore_index=True)
<ipython-input-20-9455134f582a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(blank_df,ignore_index=True)
<ipython-input-20-9455134f582a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(blank_df,ignore_index=True)
<ipython-input-20-9455134f582a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(blank_df,ignore_index=True)
<ipython-input-20-9455134f582a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pan

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
# import scipy as sc




# def butter_bandpass(lowcut, highcut, fs, order=5):
#     nyq = 0.5 * fs
#     low = lowcut / nyq
#     high = np.min([1-1/fs,highcut / nyq])
#     b, a = sc.signal.butter(order, [low, high], btype='band')
#     return b, a

# def butter_bandpass_filter(data, lowcut, highcut, fs, order=2):

#     b, a = butter_bandpass(lowcut, highcut, fs, order=order)
#     data = sc.signal.filtfilt(b, a, data)

#     b_notch,a_notch = sc.signal.iirnotch(50, 20, fs=fs)
#     data = sc.signal.filtfilt(a_notch, a_notch, data)
#     data = data-np.mean(data)
#     return data

# def filter_for_PSCs(abf, lowcut, highcut,):
#     time = abf.sweepX
#     response = abf.sweepY
#     holding = abf.sweepC
#     fs=abf.sampleRate
#     response_f1 = butter_bandpass_filter(response, lowcut, highcut, fs  )
#     response_f2 = butter_bandpass_filter(response, .01, 5000, fs  )

#     fig,ax = plt.subplots(1,figsize=[16,3])
#     ax.plot(time,response_f2,'k')
#     ax.plot(time,response_f1,'r')
#     x0 = np.random.choice(time,1)
#     x0 = 25.25
#     dur = 3
#     xlims = x0 +np.array([0,dur])
#     ax.set_xlim(xlims)
#     ax.set_ylim([-30,10])

#     return response_f1

# def find_events(response_f1,fs, sd_thresh = 5,cool_down_time=0,to_plot= True):
#     sd = np.std(response_f1)
#     print(str(sd_thresh)+'sd:',sd*sd_thresh)
#     cool_down_idx = cool_down_time*fs
#     is_event = abs(response_f1)>sd*sd_thresh
#     event_start = np.diff(is_event,prepend=0)>0
#     starts = np.where(event_start)[0]

#     x0t = -.005
#     x1t = 0.02
#     x0n = x0t*fs
#     x1n = x1t*fs

#     fig,ax = plt.subplots(1,figsize=[3,3])
#     time_wind = np.arange(x0t,x1t,1/fs)
#     stack = []
#     for e in starts:
#         indx_wind = e + np.arange(x0n,x1n,dtype=int)
#         try:
#             resp_wind = response_f1[indx_wind]
#             ax.plot(time_wind,resp_wind)
#             stack.append(resp_wind)
#         except: None

#     ax.axhline(sd*sd_thresh,color='k')
#     ax.axhline(-sd*sd_thresh,color='k')
#     stack_np = np.stack(stack,axis=0)
#     print(stack_np.shape)

#     return stack_np

# to_plot= True
# abf=abf_or_name('my_ephys_data_Today/2023x04x12_E4KI_F_P236_s001_c001_DG_0007.abf')



# response_f1 = filter_for_PSCs(abf, lowcut = 1, highcut = 500)
# stack_np = find_events(response_f1,fs, sd_thresh = 5,cool_down_time = 0.005 , to_plot= True)

# U, s, Vh = sc.linalg.svd(stack_np,full_matrices=False)
# fig,ax=plt.subplots(3)
# _ = ax[0].plot(np.arange(len(s))+1,1-(s/np.sum(s)),'o')
# ax[0].set_ylim(0,1)
# ax[0].set_xscale('log')
# u, s, vh = sc.sparse.linalg.svds(stack_np,k=1)
# _ =  ax[1].plot(vh.T @ np.diag(s) )


# stack_reduced = u @ np.diag(s) @ vh
# # _ =  ax[2].plot(stack_reduced.T)


# template = vh.T.flatten()
# template = template/np.sum(template)
# response_f1 = response_f1


# x0 = np.random.choice(time,1)
# x0 = 24.25
# dur = 4

# time = abf.sweepX
# t_ind = np.logical_and( time>=x0, time<x0+dur )

# fig,ax = plt.subplots(1)
# ax.plot(template)
# corr = np.correlate(template, response_f1 ,mode='same')

# fig,ax = plt.subplots(1,figsize=[16,3])
# ax.plot(time,response_f1,'k')
# ax.plot(time,corr,'r')
# # ax.plot(corr,'r')
# xlims = x0 +np.array([0,dur])
# ax.set_xlim(xlims)
# # ax.set_ylim([-100,10])
